In [1]:
from utils import TokenQuestion, Word2VecSolver, KenLMSolver, CrawlerSolver, RetrievalSolver, Question, Solver
import kenlm
import string
import collections
import jieba
import pandas as pd
import numpy as np
import subprocess
from subprocess import PIPE
import shlex
import re
import csv
from time import time


# def match_words(sent, opts):
    
#     words = sent.split(' ')
#     for word in words:
#         poses.append(len(word)+poses[-1])
#     sent_joined = ''.join(words)

#     for opt in opts:
#         start = sent_joined.find(opt)
#         end = start + len(opt)
#         yield words[poses.index(start):poses.index(end)]
   
# sent = '個人 覺得 natural 音波 淨 膚 儀 店 裡 的 擺設 屬於 鄉村 風 , 相當 有 味道 , 往往 一 個 很 普通 的 東西 混 著 擺 起來 就是 特別 好看 , 但 其實 試想 , 如果 把 任何 一 樣 東西 拿到 類似 辦公室 這 種 冰冷 色調 的 地方 擺 , 那 氛圍 很 容易 完全 走樣 , 實物 看起來 也 就 完全 不 生火 了 耶 ! ! ! 除了 東西 本身 , 整個 家 裡 的 色調 跟 氣氛 很 重要 阿 , 不然 再 有 風格 的 東西 拿回家 都 不 夠力 的 , 居家 裝潢 、 室 內 設計 就 是 這麼 回 事'
# opt = '音波淨膚儀'
# list(match_words(sent, [opt]))

def reformat_gf_df(df, ref):
    data2 = [x.lower().strip() for x in list(df['option'].values)]
    q = ' and '.join(['{} == "{}"'.format(x, y) for x, y in zip('abcde', data2)])
    try: 
        ans = list(ref.query(q)['ans'].values)
        if len(ans) == 0:
#             print(q)
            return None
    except:
#         print(data2, q)
        return None
    content = None
    for i, x in enumerate(data2):
        s = df.iloc[i]['content'].lower().strip()
        m = re.search(r'\s?'.join([_x for _x in x]), s)
        if m:
            pat = m.group()
            data2[i] = pat
            if not content:
                content = s.replace(pat, '︽⊙＿⊙︽') # ︽⊙＿⊙︽
        else:
#             print('!!',s, x)
            return None

    if not content:
#         print(df.iloc[0]['content'], data2, q)
        return None
    data1 = [df.iloc[0]['no'], content]
#     print(data1 + data2 + ans)
    try: new_df = pd.Series(data1 + data2 + ans, index = ['no', 'content', 'a', 'b', 'c', 'd', 'e', 'ans'])
    except: 
        print(data1 + data2 + ans)
        return None
    return new_df

def reformat_df_test(df, ref):
    pass

ANS = ['a', 'b', 'c', 'd', 'e']
WINDOW = 10
VEC_SIZE = 100
CATEGORY = {'國內旅遊': 'local_travel', '國外旅遊': 'global_travel', 
            '時尚流行': 'fashion', '美味食記': 'food', '美容彩妝': 'makeup', 'unkown': 'unkown'}

In [2]:
jieba.set_dictionary('data/DICT_CK+jieba_lower')
jieba.load_userdict('data/zhwiki-zh-clean')
jieba.add_word('龐燮傍謝', freq=10, tag='xx')


Building prefix dict from /Users/tripper/Desktop/project/pixnet-nlp/data/DICT_CK+jieba_lower ...
Loading model from cache /var/folders/4_/b_9vfbwx41g4t48jrzr0r4vm0000gn/T/jieba.uaab2b61a375629a86d8bcbcdd9e861b1.cache
Loading model cost 1.214 seconds.
Prefix dict has been built succesfully.


In [12]:
grep = RetrievalSolver('data/content-zh-lower.txt')

In [2]:
crawler = CrawlerSolver(max_query_len=20)

In [2]:
kenlm_gf_o5 = KenLMSolver('lm-experiment/model/', 'gf-lm5.barpa')

In [2]:
kenlm_uni_o5 = KenLMSolver('lm-experiment/model/', 'uni-lm5.barpa')

In [3]:
kenlm_uni_o6 = KenLMSolver('lm-experiment/model/', 'uni-lm6.barpa')

In [3]:
kenlm_jieb_o5 = KenLMSolver('lm-experiment/model/', 'jb-lm5.barpa')

In [3]:
w2v_jieba_cbow = Word2VecSolver('w2v-experiment/model/', 'jieba-cbow')

In [2]:
w2v_uni_cbow = Word2VecSolver('w2v-experiment/model/', 'uni-cbow')

In [2]:
w2v_gf_cbow = Word2VecSolver('w2v-experiment/model/', 'gf-cbow')

In [2]:
w2v_ptt_cbow = Word2VecSolver('w2v-experiment/model/', 'ptt-cbow')

In [3]:
val_set = pd.read_csv('question_sample/pre/raw_samples.csv')
test_set = pd.read_csv('question_sample/official/hackathon_1000_lower.tsv', sep='\t', names=['no','content', 'ans', 'a', 'b', 'c', 'd', 'e', 'ans_ref', 'url', 'level'], quoting=csv.QUOTE_NONE)

# kenlm(guofoo)

In [ ]:
t = time()
correct, total = 0, 0
for index, row in val_set.iterrows():
    q = TokenQuestion(row, 'guofoo')
    kenlm_gf_o5.solve(q)
    if kenlm_gf_o5.prediction != Solver.UNSOLVED:
        if kenlm_gf_o5.prediction == q.ans: correct += 1
        total += 1

print('validation accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total validation samples: {}'.format(total))

temp = total
correct, total = 0, 0
for index, row in test_set.iterrows():
    q = TokenQuestion(row, 'jieba')
    kenlm_jieb_o5.solve(q)
    if kenlm_jieb_o5.prediction != Solver.UNSOLVED:
        if kenlm_jieb_o5.prediction == q.ans: correct += 1
        total += 1

elapsed = time() - t

print('test accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total test samples: {}'.format(total))
print('elapsed time: {:.2f} sec'.format(elapsed))
print('rate: {:.2f} sec/sample'.format(float(elapsed) / float(temp + total)))

# validation accuracy: 93.82%
# total validation samples: 550
# test accuracy: 85.80%
# total test samples: 1000
# elapsed time: 55.59 sec
# rate: 0.04 sec/sample

# word2vec(gf+cbow)

In [30]:
t = time()
correct, total = 0, 0
gf_cached_pred = pd.read_csv('cached/gf_validation.tsv', sep='\t', names=['no','content', 'tag', 'option'])
gf_cached_pred.fillna('', inplace=True)
num = int(gf_cached_pred.shape[0] / 5)
for i in range(num):
    row = reformat_gf_df(gf_cached_pred[i * 5: i * 5 + 5], val_set)
    if not type(row) == pd.core.series.Series: continue
    q = TokenQuestion(row, tokenizer = 'guofoo')
    w2v_gf_cbow.solve(q, solver='syn1neg')
    if w2v_gf_cbow.prediction != Solver.UNSOLVED:
        if w2v_gf_cbow.prediction == q.ans: correct += 1
        total += 1
print('validation accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total validation samples: {}'.format(total))


temp = total
correct, total = 0, 0
gf_cached_pred = pd.read_csv('cached/gf_test.tsv', sep='\t', names=['no','content', 'tag', 'option'])
gf_cached_pred.fillna('', inplace=True)
num = int(gf_cached_pred.shape[0] / 5)

for i in range(num):
    row = reformat_gf_df(gf_cached_pred[i * 5: i * 5 + 5], test_set)
    if not type(row) == pd.core.series.Series: continue
    q = TokenQuestion(row, tokenizer = 'guofoo')
    w2v_gf_cbow.solve(q, solver='syn1neg')
    if w2v_gf_cbow.prediction != Solver.UNSOLVED:
        if w2v_gf_cbow.prediction == q.ans: correct += 1
        total += 1

elapsed = time() - t

print('test accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total test samples: {}'.format(total))
print('elapsed time: {:.2f} sec'.format(elapsed))
print('rate: {:.2f} sec/sample'.format(float(elapsed) / float(temp + total)))


# validation accuracy: 76.56%
# total validation samples: 546
# test accuracy: 76.61%
# total test samples: 945
# elapsed time: 12.45 sec
# rate: 0.01 sec/sample

validation accuracy: 76.56%
total validation samples: 546
test accuracy: 76.61%
total test samples: 945
elapsed time: 12.45 sec
rate: 0.01 sec/sample


# word2vec(ptt+cbow)

In [4]:
t = time()
correct, total = 0, 0

temp = total
# correct, total = 0, 0
ptt_cached_pred = pd.read_csv('cached/ptt_test.csv', sep='\t', names=['no','content', 'ans', 'a', 'b', 'c', 'd', 'e', 'ans_ref', 'url', 'level'], quoting=csv.QUOTE_NONE)
ptt_cached_pred.fillna('', inplace=True)
for (i1, r1), (i2, r2) in zip(test_set.iterrows(), ptt_cached_pred.iterrows()):
    q = TokenQuestion(r1, tokenizer = 'cached', cached = r2.content.strip())
    w2v_ptt_cbow.solve(q)
    if w2v_ptt_cbow.prediction != Solver.UNSOLVED:
        if w2v_ptt_cbow.prediction == q.ans: correct += 1
        total += 1
elapsed = time() - t

print('test accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total test samples: {}'.format(total))
print('elapsed time: {:.2f} sec'.format(elapsed))
print('rate: {:.2f} sec/sample'.format(float(elapsed) / float(temp + total)))

# test accuracy: 74.40%
# total test samples: 1000
# elapsed time: 1.33 sec
# rate: 0.00 sec/sample

test accuracy: 74.40%
total test samples: 1000
elapsed time: 1.33 sec
rate: 0.00 sec/sample


# kenlm(unigram o6)

In [5]:
t = time()
correct, total = 0, 0
for index, row in val_set.iterrows():
    q = TokenQuestion(row, 'uni')
    kenlm_uni_o6.solve(q)
    if kenlm_uni_o6.prediction != Solver.UNSOLVED:
        if kenlm_uni_o6.prediction == q.ans: correct += 1
        total += 1

print('validation accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total validation samples: {}'.format(total))

temp = total
correct, total = 0, 0
for index, row in test_set.iterrows():
    q = TokenQuestion(row, 'uni')
    kenlm_uni_o6.solve(q)
    if kenlm_uni_o6.prediction != Solver.UNSOLVED:
        if kenlm_uni_o6.prediction == q.ans: correct += 1
        total += 1

elapsed = time() - t

print('test accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total test samples: {}'.format(total))
print('elapsed time: {:.2f} sec'.format(elapsed))
print('rate: {:.2f} sec/sample'.format(float(elapsed) / float(temp + total)))

# validation accuracy: 98.36%
# total validation samples: 550
# test accuracy: 93.00%
# total test samples: 1000
# elapsed time: 1.53 sec
# rate: 0.00 sec/sample

validation accuracy: 98.36%
total validation samples: 550
test accuracy: 93.00%
total test samples: 1000
elapsed time: 1.53 sec
rate: 0.00 sec/sample


# kenlm(jieba)

In [5]:
# kenlm(jieba)
t = time()
correct, total = 0, 0
for index, row in val_set.iterrows():
    q = TokenQuestion(row, 'jieba')
    kenlm_jieb_o5.solve(q)
    if kenlm_jieb_o5.prediction != Solver.UNSOLVED:
        if kenlm_jieb_o5.prediction == q.ans: correct += 1
        total += 1

print('validation accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total validation samples: {}'.format(total))

temp = total
correct, total = 0, 0
for index, row in test_set.iterrows():
    q = TokenQuestion(row, 'jieba')
    kenlm_jieb_o5.solve(q)
    if kenlm_jieb_o5.prediction != Solver.UNSOLVED:
        if kenlm_jieb_o5.prediction == q.ans: correct += 1
        total += 1

elapsed = time() - t

print('test accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total test samples: {}'.format(total))
print('elapsed time: {:.2f} sec'.format(elapsed))
print('rate: {:.2f} sec/sample'.format(float(elapsed) / float(temp + total)))

# validation accuracy: 93.82%
# total validation samples: 550
# test accuracy: 85.80%
# total test samples: 1000
# elapsed time: 55.59 sec
# rate: 0.04 sec/sample

validation accuracy: 93.82%
total validation samples: 550
test accuracy: 85.80%
total test samples: 1000
elapsed time: 55.59 sec
rate: 0.04 sec/sample


# crawler retrieval engine

In [4]:
t = time()
cached_file = 'cached/crawler_validation.csv'
pred = pd.DataFrame(columns=['no', 'prediction'])
correct, total = 0, 0
for index, row in val_set.iterrows():
    q = TokenQuestion(row, 'uni')
    crawler.solve(q)
    pred = pred.append({'no': q.no, 'prediction': crawler.prediction}, ignore_index = True)
    if crawler.prediction != Solver.UNSOLVED:
        if crawler.prediction == q.ans: correct += 1
        total += 1
pred['no'] = pred['no'].astype(int)
pred.to_csv(cached_file, index=False)
print('validation accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total validation samples: {}'.format(total))

temp = total
cached_file = 'cached/crawler_test.csv'
pred = pd.DataFrame(columns=['no', 'prediction'])
correct, total = 0, 0
for index, row in test_set.iterrows():
    q = TokenQuestion(row, 'uni')
    crawler.solve(q)
    pred = pred.append({'no': q.no, 'prediction': crawler.prediction}, ignore_index = True)
    if crawler.prediction != Solver.UNSOLVED:
        if crawler.prediction == q.ans: correct += 1
        total += 1
elapsed = time() - t
pred['no'] = pred['no'].astype(int)
pred.to_csv(cached_file, index=False)

print('test accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total test samples: {}'.format(total))
print('elapsed time: {:.2f} sec'.format(elapsed))
print('rate: {:.2f} sec/sample'.format(float(elapsed) / float(temp + total)))

# validation accuracy: 77.64%
# total validation samples: 550
# test accuracy: 62.00%
# total test samples: 1000
# elapsed time: 5743.00 sec
# rate: 3.71 sec/sample

validation accuracy: 77.64%
total validation samples: 550
test accuracy: 62.00%
total test samples: 1000
elapsed time: 5743.00 sec
rate: 3.71 sec/sample


In [7]:
# kenlm(unigram-o5)
t = time()
correct, total = 0, 0
for index, row in val_set.iterrows():
    q = TokenQuestion(row, 'uni')
    kenlm_uni_o5.solve(q)
    if kenlm_uni_o5.prediction != Solver.UNSOLVED:
        if kenlm_uni_o5.prediction == q.ans: correct += 1
        total += 1

print('validation accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total validation samples: {}'.format(total))

temp = total
correct, total = 0, 0
for index, row in test_set.iterrows():
    q = TokenQuestion(row, 'uni')
    kenlm_uni_o5.solve(q)
    if kenlm_uni_o5.prediction != Solver.UNSOLVED:
        if kenlm_uni_o5.prediction == q.ans: correct += 1
        total += 1

elapsed = time() - t

print('test accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total test samples: {}'.format(total))
print('elapsed time: {:.2f} sec'.format(elapsed))
print('rate: {:.2f} sec/sample'.format(float(elapsed) / float(temp + total)))

# validation accuracy: 98.18%
# total validation samples: 550
# test accuracy: 92.90%
# total test samples: 1000
# elapsed time: 5.67 sec
# rate: 0.00 sec/sample

validation accuracy: 98.18%
total validation samples: 550
test accuracy: 92.90%
total test samples: 1000
elapsed time: 5.67 sec
rate: 0.00 sec/sample


# direct word2vec(unigram+cbow)

In [7]:
t = time()
correct, total = 0, 0
for index, row in val_set.iterrows():
    q = TokenQuestion(row, 'uni')
    w2v_uni_cbow.solve(q, uni_gram = True)
    if w2v_uni_cbow.prediction != Solver.UNSOLVED:
        if w2v_uni_cbow.prediction == q.ans: correct += 1
        total += 1
    
print('validation accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total validation samples: {}'.format(total))

temp = total
correct, total = 0, 0
for index, row in test_set.iterrows():
    q = TokenQuestion(row, 'uni')
    w2v_uni_cbow.solve(q, uni_gram = True)
    if w2v_uni_cbow.prediction != Solver.UNSOLVED:
        if w2v_uni_cbow.prediction == q.ans: correct += 1
        total += 1

elapsed = time() - t

print('test accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total test samples: {}'.format(total))
print('elapsed time: {:.2f} sec'.format(elapsed))
print('rate: {:.2f} sec/sample'.format(float(elapsed) / float(temp + total)))

# validation accuracy: 63.19%
# total validation samples: 546
# test accuracy: 59.60%
# total test samples: 1000
# elapsed time: 2.32 sec
# rate: 0.00 sec/sample

validation accuracy: 63.19%
total validation samples: 546
test accuracy: 59.60%
total test samples: 1000
elapsed time: 2.32 sec
rate: 0.00 sec/sample


# grep + word2vec(jieba+cbow) aggregation

In [15]:
t = time()
correct, total = 0, 0
grep_cached_pred = pd.read_csv('cached/grep_validation.csv')
grep_cached_pred.fillna('', inplace=True)
for (i1, r1), (i2, r2) in zip(val_set.iterrows(), grep_cached_pred.iterrows()):
    q = Question(r1)
    if r2.prediction != str(Solver.UNSOLVED):
        if r2.prediction == q.ans: correct += 1
        elif len(r2.key) > 0:
            w2v_jieba_cbow.solve(q, key = r2.key)
            if w2v_jieba_cbow.prediction == q.ans: correct += 1
        total += 1
print('validation accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total validation samples: {}'.format(total))

temp = total
correct, total = 0, 0
grep_cached_pred = pd.read_csv('cached/grep_test.csv')
grep_cached_pred.fillna('', inplace=True)
for (i1, r1), (i2, r2) in zip(test_set.iterrows(), grep_cached_pred.iterrows()):
    q = Question(r1)
    if r2.prediction != str(Solver.UNSOLVED):
        if r2.prediction == q.ans: correct += 1
        elif len(r2.key) > 0:
            w2v_jieba_cbow.solve(q, key = r2.key)
            if w2v_jieba_cbow.prediction == q.ans: correct += 1
        total += 1
elapsed = time() - t

print('test accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total test samples: {}'.format(total))
print('elapsed time: {:.2f} sec'.format(elapsed))
print('rate: {:.2f} sec/sample'.format(float(elapsed) / float(temp + total)))

# validation accuracy: 93.77%
# total validation samples: 546
# test accuracy: 86.80%
# total test samples: 1000
# elapsed time: 561.27 sec
# rate: 0.36 sec/sample

validation accuracy: 93.77%
total validation samples: 546
test accuracy: 86.80%
total test samples: 1000
elapsed time: 1.25 sec
rate: 0.00 sec/sample


# direct word2vec(jieba+cbow)

In [8]:
t = time()
correct, total = 0, 0
for index, row in val_set.iterrows():
    q = TokenQuestion(row, 'jieba')
    w2v_jieba_cbow.solve(q, solver = 'both')
    if w2v_jieba_cbow.prediction != Solver.UNSOLVED:
        if w2v_jieba_cbow.prediction == q.ans: correct += 1
        total += 1
print('validation accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total validation samples: {}'.format(total))

temp = total
correct, total = 0, 0
for index, row in test_set.iterrows():
    q = TokenQuestion(row, 'jieba')
    w2v_jieba_cbow.solve(q, solver = 'both')
    if w2v_jieba_cbow.prediction != Solver.UNSOLVED:
        if w2v_jieba_cbow.prediction == q.ans: correct += 1
        total += 1

elapsed = time() - t

print('test accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total test samples: {}'.format(total))
print('elapsed time: {:.2f} sec'.format(elapsed))
print('rate: {:.2f} sec/sample'.format(float(elapsed) / float(temp + total)))

# == solver = syn1neg
# validation accuracy: 79.85%
# total validation samples: 546
# test accuracy: 79.70%
# total test samples: 1000
# elapsed time: 2.48 sec
# rate: 0.00 sec/sample

# == solver = syn0
# validation accuracy: 65.75%
# total validation samples: 546
# test accuracy: 66.20%
# total test samples: 1000
# elapsed time: 2.61 sec
# rate: 0.00 sec/sample

validation accuracy: 79.30%
total validation samples: 546
test accuracy: 78.60%
total test samples: 1000
elapsed time: 2.37 sec
rate: 0.00 sec/sample


# grep retrieval engine

In [16]:
t = time()
cached_file = 'cached/grep_validation.csv'
pred = pd.DataFrame(columns=['no', 'key', 'prediction'])
correct, total = 0, 0
for index, row in val_set.iterrows():
    q = Question(row)
    grep.solve(q)
    pred = pred.append({'no': q.no, 'key': grep.key, 'prediction': grep.prediction}, ignore_index = True)
    if grep.prediction != Solver.UNSOLVED:
        if grep.prediction == q.ans: correct += 1
        total += 1
pred['no'] = pred['no'].astype(int)
pred.to_csv(cached_file, index=False)
print('validation accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total validation samples: {}'.format(total))

temp = total
cached_file = 'cached/grep_test.csv'
pred = pd.DataFrame(columns=['no', 'key', 'prediction'])
correct, total = 0, 0
for index, row in test_set.iterrows():
    q = Question(row)
    grep.solve(q)
    pred = pred.append({'no': q.no, 'key': grep.key, 'prediction': grep.prediction}, ignore_index = True)
    if grep.prediction != Solver.UNSOLVED:
        if grep.prediction == q.ans: correct += 1
        total += 1
elapsed = time() - t
pred['no'] = pred['no'].astype(int)
pred.to_csv(cached_file, index=False)

print('test accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total test samples: {}'.format(total))
print('elapsed time: {:.2f} sec'.format(elapsed))
print('rate: {:.2f} sec/sample'.format(float(elapsed) / float(temp + total)))

# validation accuracy: 93.77%
# total validation samples: 546
# test accuracy: 71.40%
# total test samples: 1000
# elapsed time: 482.07 sec
# rate: 0.31 sec/sample

validation accuracy: 93.77%
total validation samples: 546
test accuracy: 71.40%
total test samples: 1000
elapsed time: 4811.74 sec
rate: 3.11 sec/sample
